IMDB Movies Reviews SENTIMENT ANALYSIS

In [6]:
import numpy as np
import pandas as pd

In [7]:
#loading dataset
data= pd.read_csv("dataset/IMDB Dataset.csv")

In [ ]:
#print first five rows of dataset
data.head()

In [9]:
# names of columns
data.columns

Index(['review', 'sentiment'], dtype='object')

In [13]:
#check for null values
data.isnull().any()

review       False
sentiment    False
dtype: bool

## Text Normalization
### Tokenization

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

In [14]:
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
#tokenization
tokens= ToktokTokenizer()
stopwords= nltk.corpus.stopwords.words('english')

#### noise removal
using BeautifulSoup for removing any html tags that might be present

In [12]:
# noise removal
def noise_removal(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub('\[[^]]*\]', '', text)
    return text

<>:5: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\['
C:\Users\HP\AppData\Local\Temp\ipykernel_12200\1029199373.py:5: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[[^]]*\]', '', text)


In [15]:
data['review'] = data['review'].apply(noise_removal)

C:\Users\HP\AppData\Local\Temp\ipykernel_12200\1029199373.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


### Stemming

In [18]:
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [19]:
data['review'] = data['review'].apply(stemmer)

In [21]:
# check if stemming is done
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


### Removing Stopwords

In [22]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 

In [23]:
# set stop words to English
stop_wr=set(stopwords.words('english'))
print(stop_wr)

{'doing', 'how', 'themselves', 'wouldn', "that'll", 'few', 'that', 'hers', 'theirs', 'can', 'above', 'more', 'haven', 'his', "doesn't", 'ourselves', "didn't", 'myself', "mightn't", 'while', 'too', "won't", 'him', 'mustn', 'are', 'hadn', "mustn't", 'no', 'should', 'same', 'aren', 'before', 'all', "she's", 'which', 'off', 'through', 'then', 'nor', 'been', "you'd", "couldn't", 'after', 'be', 'from', 'at', "needn't", 'during', 'ours', "you'll", 'such', 'up', 've', 'once', 'its', 'i', "don't", 'my', 'himself', 'she', "wasn't", 'or', 'to', 'by', "hasn't", 'why', 'her', 'with', 'both', 'herself', 'between', 'into', 'didn', 'needn', 'a', 'itself', "you're", 'me', 'has', 'there', "it's", 'those', 'against', 'again', 'an', 'but', 'o', 'because', 'than', 'couldn', "you've", 'your', 'were', 'when', 'y', 'weren', 'shan', 'does', "hadn't", 'some', 'd', 'of', 'whom', 'wasn', 'these', 'only', 'own', "aren't", 'as', 'yourself', 'what', 'isn', 'our', 'it', 'is', 'am', "should've", 'was', 'we', 'on', "wo

In [24]:
#removing the stopwords
def removing_stopwords(text, is_lower_case=False):  #set lower case flag to false 
    #Tokenization of text
    tokenizers=ToktokTokenizer()
    #Setting English stopwords
    tokens = tokenizers.tokenize(text)
    tokens = [token.strip() for token in tokens]    #remove whitespace
    if is_lower_case:
        filter_tokens = [token for token in tokens if token not in stop_wr] #if text in lowercase then compare to stopwords
    else:
        filter_tokens = [token for token in tokens if token.lower() not in stop_wr] #if not in lowercase then convert and compare
    filtered_text = ' '.join(filter_tokens)    
    return filtered_text

In [25]:
data['review'] = data['review'].apply(removing_stopwords)

In [26]:
data.head()

,review,sentiment
0,one review ha mention watch 1 oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive
